In [1]:
!pip install rouge-score

  Preparing metadata (setup.py) ... - done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=a581cd23c32db7cea55f03c0950694c653201770c7ebc6ea34c09c183aec814b
  Stored in directory: /root/.cache/pip/wheels/8e/6b/70/59daa7c90a238610e34bac5916e001fe3d9bb0ec59c8cf5518
Successfully built rouge-score


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

from transformers import AutoTokenizer
from datasets import load_dataset, load_metric
    
model_checkpoint = "t5-small"
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

prefix = "summarize: "

metric = load_metric("rouge")

batch_size = 16

max_input_length = 1024
max_target_length = 184


In [4]:
class SummaryDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = dataframe['article']
        self.summarytext = dataframe['abstract']

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        
        text = 'summarize: ' +str(self.text[index])
        text = ' '.join(text.split())

        summarytext = str(self.summarytext[index])
        #summarytext = 'summarize: ' +str(self.summarytext[index])
        summarytext = ' '.join(summarytext.split())

        source = self.tokenizer.batch_encode_plus([text], max_length= self.source_len, pad_to_max_length=True,truncation=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([summarytext], max_length= self.summ_len, pad_to_max_length=True,truncation=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [5]:
test_dataset = load_dataset("ccdv/arxiv-summarization",'document',split="test")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset arxiv_summarization_dataset downloaded and prepared to /root/.cache/huggingface/datasets/ccdv___arxiv_summarization_dataset/document/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3. Subsequent calls will reuse this data.


In [6]:
test_dataset

Dataset({
    features: ['article', 'abstract'],
    num_rows: 6440
})

In [7]:
test_set = SummaryDataset(test_dataset, tokenizer, max_input_length, max_target_length)

In [8]:
test_params = {
        'batch_size': batch_size ,
        'shuffle': False,
        'num_workers': 0
        }

In [9]:
test_loader = DataLoader(test_set, **test_params)

In [10]:
def predict(tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=max_target_length 
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [11]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
modelpath = '/kaggle/input/scientificpapers-arxiv-sample-summarization/t5-mini-arxiv.pt'
model = AutoModelForSeq2SeqLM.from_pretrained(modelpath)
model = model.to(device)

In [12]:
predictions, actuals = predict(tokenizer, model, device, test_loader)

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2352: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Completed 0
Completed 100
Completed 200
Completed 300
Completed 400


In [13]:
final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
final_df.to_csv('predictions.csv')

In [14]:
result = metric.compute(predictions=predictions, references=actuals, use_stemmer=True)
# Extract a few results
result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

result

{'rouge1': 36.923061498737674,
 'rouge2': 11.689969991045999,
 'rougeL': 23.694148274627004,
 'rougeLsum': 23.697016690655214}

In [15]:
# References
# https://github.com/huggingface/notebooks/blob/main/examples/summarization.ipynb
#https://colab.research.google.com/github/abhimishra91/transformers-tutorials/blob/master/transformers_summarization_wandb.ipynb
